In [1]:
import pickle
with open('../input/pickles/train.pickle', 'rb') as f:
    X_train, y_train = pickle.load(f)

In [2]:
with open('../input/pickles/test.pickle', 'rb') as f:
    X_test, y_test = pickle.load(f)

In [3]:
print (X_train.shape, y_train.shape)

(49262, 64, 64, 3) (49262, 37)


In [4]:
ORIG_SHAPE = (424,424)
CROP_SIZE = (256,256)
IMG_SHAPE = (64,64)



from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D
from keras import backend as K
import datetime

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
    

model = Sequential()
model.add(Conv2D(512, (3, 3), input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
model.add(Conv2D(256, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(GlobalMaxPooling2D())


model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(37))
model.add(Activation('sigmoid'))
    



log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=[root_mean_squared_error])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 512)       14336     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 256)       1179904   
_________________________________________________________________
activation (Activation)      (None, 60, 60, 256)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 256)       590080    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 128)       295040    
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 128)       0

In [5]:
%load_ext tensorboard
%tensorboard --logdir logs


In [6]:
batch_size = 32
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size,callbacks=[tensorboard_callback])

Epoch 1/30
1540/1540 [==============================] - 93s 60ms/step - loss: 0.3110 - root_mean_squared_error: 0.1721 - val_loss: 0.2956 - val_root_mean_squared_error: 0.1591
Epoch 2/30
1540/1540 [==============================] - 91s 59ms/step - loss: 0.2891 - root_mean_squared_error: 0.1529 - val_loss: 0.2694 - val_root_mean_squared_error: 0.1335
Epoch 3/30
1540/1540 [==============================] - 91s 59ms/step - loss: 0.2686 - root_mean_squared_error: 0.1332 - val_loss: 0.2590 - val_root_mean_squared_error: 0.1226
Epoch 4/30
1540/1540 [==============================] - 91s 59ms/step - loss: 0.2613 - root_mean_squared_error: 0.1251 - val_loss: 0.2537 - val_root_mean_squared_error: 0.1161
Epoch 5/30
1540/1540 [==============================] - 91s 59ms/step - loss: 0.2572 - root_mean_squared_error: 0.1202 - val_loss: 0.2510 - val_root_mean_squared_error: 0.1124
Epoch 6/30
1540/1540 [==============================] - 91s 59ms/step - loss: 0.2541 - root_mean_squared_error: 0.1163 -

In [7]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 108), started 0:49:16 ago. (Use '!kill 108' to kill it.)

In [8]:
model.save('30epochsslow3).h5')


In [9]:
!mkdir -p saved_model
model.save('saved_model/30epochsslow3')

In [10]:
model.save_weights('saved_weightsslow3')
